# Preguntas
- Cuánto pagan de expensas los departamentos en CABA según barrio?
  - La media de las expensas según barrio
  - Media del ratio entre alquiler y expensas
  - media de expensas por m2

Para ello:
- Método para detectar outliers

In [5]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import re

from config.config_jupyter import set_wd
set_wd()
from src.zonaprop_wrangling import main_zonaprop_wrangling

c:\Users\marti\OneDrive\Documentos\Trabajos Indec\Scrap alquileres caba


In [6]:
df = pd.read_parquet("output/zonaprop_vivienda.pkt")
df = main_zonaprop_wrangling(df)

In [7]:
df.value_counts("currency")

currency
ARS          4883
USD          4363
Consultar     149
Name: count, dtype: int64

In [8]:
df_ars = df.copy()
df_ars = df_ars[df_ars["currency"] == "ARS"].reset_index(drop = True)
df_ars["Expensas / alquiler"] = df_ars["Expensas"] / df_ars["Price"]
df_ars["Expensas / m2"] = df_ars["Expensas"] / df_ars["metros_cuadrados"]

In [9]:
df_usd = df.copy()
df_usd = df_usd[df_usd["currency"] == "USD"].reset_index(drop = True)

In [10]:
df_ars["Expensas"].isna().sum()

689

In [11]:
df_ars = df_ars[
                (df_ars["Expensas"]<=200_000)
                & (df_ars["Expensas"].notna())
                & (df_ars["Expensas"]>=10_000)
                & (df_ars["Price"]<=600_000)
                & (df_ars["Price"]>=10_000)
                & (df_ars["Expensas / alquiler"]<=1)
                & (df_ars["Expensas / m2"]<=10_000)
                & (df_ars["Expensas / m2"]>=10)
                ]
df_ars = df_ars.sort_values("Location")

In [12]:
df_ars.sort_values("Expensas / m2").iloc[-2,7]

'https://www.zonaprop.com.ar/propiedades/clasificado/alclapin--amob-y-equip-mansilla-al-2800-frente-26-m-sup2-53026764.html'

In [14]:
fig = go.Figure()
fig.add_trace(go.Box(x = df_ars["Location"],y = df_ars["Expensas"]))
fig.update_layout(template = None, height = 600, margin = {'b':250})
fig.update_xaxes(type = "category")

In [15]:
fig = go.Figure()
fig.add_trace(go.Box(x = df_ars["Location"],y = df_ars["Price"]))
fig.update_layout(template = None, height = 600, margin = {'b':250})

In [16]:
tmp_df = df_ars.copy()
tmp_df = tmp_df.groupby("Location", as_index=False).median(numeric_only = True)
fig = go.Figure()
fig.add_trace(go.Scatter(x = tmp_df["Location"], y = tmp_df["Expensas / m2"], mode = "markers"))
fig.update_layout(template = None, margin = {"b":250})

In [17]:
df_usd = df_usd[
    (df_usd["metros_cuadrados"]<=60)
    & (df_usd["metros_cuadrados"].notna())
    & (df_usd["Price"] <= 2_000)
]

In [18]:
df_usd.sort_values("Price", ascending=True)[:20].iloc[18,5]

'Monoambiente en alquiler a Estrenar en el Barrio de San Cristóbal. Ubicado en San Cristobal, este moderno monoambiente ofrece una oportunidad única de vivir en un espacio contemporáneo y funcional. Situado en el quinto piso de un edificio de nueva construcción, este departamento a estrenar garantiza una experiencia de vida cómoda y llena de estilo. Condiciones de ingreso: - 1 mes de adelanto. - 1 mes de deposito. - Seguro de caución. condiciones del contrato: - 24 meses (dos años) en dolares. Características Principales: Superficie Total: 41 metros cuadrados. Superficie Cubierta: 39 metros cuadrados. Ubicación: Quinto piso con vista al contrafrente, asegurando tranquilidad y luminosidad. Distribución: Espacio con cocina integrada para una mayor sensación de amplitud. Estado: A estrenar, lo que garantiza un ambiente impecable y moderno. Detalles Adicionales: Luminosidad: Grandes ventanales permiten la entrada abundante de luz natural, creando un ambiente cálido y acogedor. Cocina Integ

In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df_usd["Location"],y = df_usd["Price"], mode = "markers"))
fig.update_layout(template = None, height = 600, margin = {'b':250})

In [20]:
tmp_df = df_usd.copy()
tmp_df = tmp_df.groupby("Location", as_index=False).median(numeric_only = True)
fig = go.Figure()
fig.add_trace(go.Scatter(x = tmp_df["Location"], y = tmp_df["Price"], mode = "markers"))
fig.update_layout(template = None, margin = {"b":250}, title_text = "Costo de alquiler en USD. Fuente Zonaprop",
                  )

In [21]:
fig = go.Figure()
fig.add_trace(go.Box(y = df_usd["Price"], x = df_usd["Location"]))
fig.update_layout(template = None, margin = {"b":250}, title_text = "Costo de alquiler en USD. Fuente Zonaprop",
                  height = 1_000)
fig.show()